<a href="https://colab.research.google.com/github/cseveriano/evolving_clustering/blob/master/notebooks/Evolving_Clustering_Static_DS3_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
path = "/content/gdrive/My Drive/Evolving_Results/"
from google.colab import drive
drive.mount("/content/gdrive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#@title
!apt-get update
!apt-get install r-base
!pip install rpy2
!apt-get install libmagick++-dev
#!apt-get install r-cran-rjava

import os       #importing os to set environment variable
def install_java():
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null      #install openjdk
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     #set environment variable
  os.environ["LD_LIBRARY_PATH"] = "/usr/lib/jvm/java-8-openjdk-amd64/jre/lib/amd64:/usr/lib/jvm/java-8-openjdk-amd64/jre/lib/amd64/server"
  !java -version       #check java version

install_java()

!R CMD javareconf

#!apt-get install r-cran-rjava
#!apt-get install libgdal-dev libproj-dev

!R -e 'install.packages(c("magick",  "animation", "stream", "rJava", "streamMOA"))'

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,609 B]
Hit:12 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [304 kB]
Ge

##Install R Packages

In [0]:
# enables the %%R magic, not necessary if you've already done this
%load_ext rpy2.ipython

In [0]:
%%R
dyn.load("/usr/lib/jvm/java-8-openjdk-amd64/jre/lib/amd64/server/libjvm.so")
library("stream")
library("streamMOA")

/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Loading required package: proxy

  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: 
Attaching package: ‘proxy’


  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: The following objects are masked from ‘package:stats’:

    as.dist, dist


  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: The following object is masked from ‘package:base’:

    as.matrix


  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Loading required package: rJava

  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Rjava.init.warning: stack size reduced from u

##Generate Concept Drift Data Stream


*   Choose concept drift benchmark (read Bolanos paper)
*   send outputs: 
      * stream_df
      * prequential window size
     




In [0]:
%%R
experiment <- function(){
  df <- read.csv("https://query.data.world/s/faarcpz24ukr2g53druxljp4nghhzn", header=TRUE);
  nsamples <- nrow(df)
  df <- df[sample(nsamples),]
  stream <- DSD_Memory(df[,c("x", "y")], class=df[,"class"], k=max(df[,"class"]))
  return (get_points(stream, n = nsamples, class = TRUE))
}

##Run Benchmark Models

### Benchmark methods:
* DBStream
* DenStream
* ClusStream
* Stream KM++

### Benchmark metrics:
* Precision



In [0]:
# Experiment parameters
nclusters = 9
nsamples = 3000
train_size = 1800
window_size = 100
metric = "cRand"
trials = 30

In [0]:
%%R -i metric -i window_size -i trials -i path -i nclusters

alg_names <- c("DenStream", "Clustream", "StreamKM")
trials_df <- data.frame(matrix(ncol = length(alg_names), nrow = 0))
colnames(trials_df) <- alg_names

for (i in 1:(trials)){
  algorithms <- list("DenStream" = DSC_DenStream(epsilon=0.01, mu=4, beta=0.4),
                     "Clustream" = DSC_CluStream(m = 10, horizon = 10, t = 1, k=NULL),
                     "StreamKM" = DSC_StreamKM(sizeCoreset = 100, numClusters = nclusters)
  )
  writeLines(sprintf("Trial: %d", i))
    
  evaluation <- sapply(algorithms, FUN = function(alg) {
    
    df <- read.csv("https://query.data.world/s/faarcpz24ukr2g53druxljp4nghhzn", header=TRUE);
    nsamples <- nrow(df)
    df <- df[sample(nsamples),]
    stream <- DSD_Memory(df[,c("x", "y")], class=df[,"class"], k=max(df[,"class"]))
    update(alg, stream, n=nsamples)
    reset_stream(stream)
    evaluate(alg, stream, measure = metric, n = nsamples, type = "macro", assign = "macro")
  })
    
  trials_df[nrow(trials_df) + 1,] = as.data.frame(evaluation)[,'evaluation']
}

write.csv(trials_df, paste(path,"results_DS3_benchmark.csv"))

Trial: 1
Trial: 2
Trial: 3
Trial: 4
Trial: 5
Trial: 6
Trial: 7
Trial: 8
Trial: 9
Trial: 10
Trial: 11
Trial: 12
Trial: 13
Trial: 14
Trial: 15
Trial: 16
Trial: 17
Trial: 18
Trial: 19
Trial: 20
Trial: 21
Trial: 22
Trial: 23
Trial: 24
Trial: 25
Trial: 26
Trial: 27
Trial: 28
Trial: 29
Trial: 30


##Run Evolving Clustering

* Convert to X,y format
* run prequential routine
* plot results

In [0]:
!pip install -U git+https://github.com/cseveriano/evolving_clustering

  Cloning https://github.com/cseveriano/evolving_clustering to /tmp/pip-req-build-nm_vt098
  Running command git clone -q https://github.com/cseveriano/evolving_clustering /tmp/pip-req-build-nm_vt098
  Stored in directory: /tmp/pip-ephem-wheel-cache-cuwg8sao/wheels/aa/b7/f1/4d93077e1b97361934f5992c77be80f0769eba6e0e9da6e22d
Successfully built evolclustering


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from evolving import EvolvingClustering, load_dataset, Metrics, Benchmarks, util
from sklearn.metrics import adjusted_rand_score
import time
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
pandas2ri.activate()
r = robjects.r

In [0]:
evol_trials_df = pd.DataFrame(columns=["microTEDAclus"])

for i in np.arange(trials):

  named_tuple = time.localtime() # get struct_time
  time_string = time.strftime("%m/%d/%Y, %H:%M:%S", named_tuple)
  print("Trial: ",i ," at ",time_string)

  stream_df = pandas2ri.ri2py_dataframe(r.experiment())
  X = stream_df[['x', 'y']].values
  y = stream_df['class'].values
  evol_model = EvolvingClustering.EvolvingClustering(variance_limit=0.0006, debug=False)

  evol_model.fit(X)
  y_hat = evol_model.predict(X)

  error = adjusted_rand_score(y, y_hat)
  evol_trials_df = evol_trials_df.append({'microTEDAclus': error}, ignore_index=True)
  evol_trials_df.to_csv(path+'results_DS3_evolving.csv', index=False)

Trial:  0  at  05/10/2019, 14:28:14


/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)


Trial:  1  at  05/10/2019, 14:28:51


/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)


Trial:  2  at  05/10/2019, 14:29:29
Trial:  3  at  05/10/2019, 14:30:06
Trial:  4  at  05/10/2019, 14:30:43
Trial:  5  at  05/10/2019, 14:31:20
Trial:  6  at  05/10/2019, 14:31:59
Trial:  7  at  05/10/2019, 14:32:35
Trial:  8  at  05/10/2019, 14:33:13
Trial:  9  at  05/10/2019, 14:33:50
Trial:  10  at  05/10/2019, 14:34:27
Trial:  11  at  05/10/2019, 14:35:04
Trial:  12  at  05/10/2019, 14:35:42
Trial:  13  at  05/10/2019, 14:36:19
Trial:  14  at  05/10/2019, 14:36:57
Trial:  15  at  05/10/2019, 14:37:35
Trial:  16  at  05/10/2019, 14:38:12
Trial:  17  at  05/10/2019, 14:38:49
Trial:  18  at  05/10/2019, 14:39:26
Trial:  19  at  05/10/2019, 14:40:02
Trial:  20  at  05/10/2019, 14:40:40
Trial:  21  at  05/10/2019, 14:41:17
Trial:  22  at  05/10/2019, 14:41:54
Trial:  23  at  05/10/2019, 14:42:33
Trial:  24  at  05/10/2019, 14:43:10
Trial:  25  at  05/10/2019, 14:43:47
Trial:  26  at  05/10/2019, 14:44:25
Trial:  27  at  05/10/2019, 14:45:03
Trial:  28  at  05/10/2019, 14:45:40
Trial:  2